In [1]:
import pandas as pd
# Import Required Modules
from flask import Flask, render_template
import pandas as pd
import json
import plotly
import plotly.express as px
import pandas_ta as pta
from plotly.subplots import make_subplots
import plotly.graph_objects as go
from datetime import date
from datetime import timedelta, date
from prophet import Prophet
import pandas as pd
import numpy as np
import yfinance as yf
import seaborn as sns
import mpld3
import matplotlib.pyplot as plt
df = pd.read_csv('resources/expenses.csv')


In [2]:
cols = ['Food', 'Gas', 'Misc', 'Grand Total']
for col in cols:
    df[col] = df[col].replace('[\$,]', '', regex=True).astype(float)
df.head(5)

,Month,Food,Gas,Misc,Grand Total,Year
0,January,724.48,138.36,1446.26,2309.10,2020-2021
1,January,684.73,146.46,463.68,1294.87,2021-2022
2,January,1160.07,162.49,274.58,1597.14,2022-2023
3,February,798.10,249.96,534.26,1582.32,2020-2021
4,February,842.92,184.10,242.94,1269.96,2021-2022


In [3]:
import plotly.graph_objs as go
grouped = df.groupby('Year')
grouped_all_cols_avg = grouped.mean()
grouped_all_cols_avg
# create the bar trace
trace = go.Bar(x=grouped_all_cols_avg.index, y=grouped_all_cols_avg['Grand Total'])

# create the layout
layout = go.Layout(title='Years Expenses')

# create the figure object
fig = go.Figure(data=[trace], layout=layout)

# show the chart
fig.show()
yearexpenses = json.dumps(fig, cls=plotly.utils.PlotlyJSONEncoder)
grouped_all_cols_avg

,Food,Gas,Misc,Grand Total
Year,,,,
2020-2021,829.206667,146.404167,630.7550,1606.365833
2021-2022,1004.718333,148.608333,899.8425,2053.169167
2022-2023,1290.710000,188.247500,716.5250,2195.482500


In [4]:
from pandas.api.types import CategoricalDtype

# Define an ordered set of month categories
month_cats = ['January', 'February', 'March', 'April', 'May', 'June',
              'July', 'August', 'September', 'October', 'November', 'December']
cat_type = CategoricalDtype(categories=month_cats, ordered=True)

# Convert 'Month' column to categorical with Month categories
df['Month'] = df['Month'].astype(cat_type)

# Group by Year and Month, then take the mean of each group
grouped = df.groupby(['Year', 'Month']).mean()

# Sort by Year (ascending) and Month (using the specified order)
grouped = grouped.sort_index(level=[0, 1])
# calculate percentage change and add new column to the right of Grand Total
grouped['% Change'] = grouped['Grand Total'].pct_change() *100
grouped.fillna(0, inplace=True)

grouped.head(5)

Food     Gas     Misc  Grand Total   % Change
Year      Month                                                     
2020-2021 January    724.48  138.36  1446.26      2309.10   0.000000
          February   798.10  249.96   534.26      1582.32 -31.474600
          March      923.71  214.23   445.92      1583.86   0.097325
          April      917.84   75.00  1983.43      2976.27  87.912442
          May       1074.11   45.84   247.26      1367.21 -54.062971

In [5]:
# reset index, leaving out the year level
month_Avg = grouped.reset_index()
month_Avg = month_Avg.groupby(['Month']).mean()

import plotly.graph_objs as go

# create the bar trace
trace = go.Bar(x=month_Avg.index, y=month_Avg['Grand Total'])

# create the layout
layout = go.Layout(title='Average Monthly Expenses')

# create the figure object
fig = go.Figure(data=[trace], layout=layout)

# show the chart
fig.show()
month_Avg.head(5)
# month_Avg = json.dumps(fig, cls=plotly.utils.PlotlyJSONEncoder)




,Food,Gas,Misc,Grand Total,% Change
Month,,,,,
January,856.426667,149.103333,728.173333,1733.703333,-9.712583
February,977.366667,192.233333,405.756667,1575.356667,-5.358919
March,903.783333,211.296667,835.453333,1950.533333,32.486279
April,1379.886667,107.150000,1369.343333,2856.380000,62.578397
May,1284.013333,127.700000,631.193333,2042.906667,-27.123460


In [6]:
# reset the second index and move it to a regular column
result = grouped.reset_index(level=0)
result.columns = ['Year','Food', 'Gas', 'Misc', 'Grand Total', '% Change']

result.head(5)


,Year,Food,Gas,Misc,Grand Total,% Change
Month,,,,,,
January,2020-2021,724.48,138.36,1446.26,2309.10,0.000000
February,2020-2021,798.10,249.96,534.26,1582.32,-31.474600
March,2020-2021,923.71,214.23,445.92,1583.86,0.097325
April,2020-2021,917.84,75.00,1983.43,2976.27,87.912442
May,2020-2021,1074.11,45.84,247.26,1367.21,-54.062971


In [7]:
import plotly.graph_objs as go

# 2020-2021 Data
filter2020_2021 = result[result['Year'] == '2020-2021']

# Create the Plotly trace using the filtered DataFrame
trace1 = go.Bar(
    x=filter2020_2021.index,
    y=filter2020_2021['Grand Total'],
    name='Expenses'
)

# create trace for second y-axis
trace2 = go.Scatter(
    x=result.index,
    y=filter2020_2021['% Change'],
    name='% Change',
    yaxis='y2'
)

# define layout with two y-axes
layout = go.Layout(
    title='2020-2021 Expenses',
    yaxis=dict(
        title='Expenses'
    ),
    yaxis2=dict(
        title='% Change',
        overlaying='y',
        side='right'
    )
)

# create figure object
fig1 = go.Figure(data=[trace1, trace2], layout=layout)

# show the figure
fig1.show()
Expenses2020_2021 = json.dumps(fig1, cls=plotly.utils.PlotlyJSONEncoder)

In [8]:
# 2021-2022 Data
filter2021_2022 = result[result['Year'] == '2021-2022']
# Create the Plotly trace using the filtered DataFrame
trace1 = go.Bar(
    x=filter2021_2022.index,
    y=filter2021_2022['Grand Total'],
    name='Expenses'
)

# create trace for second y-axis
trace2 = go.Scatter(
    x=result.index,
    y=filter2021_2022['% Change'],
    name='% Change',
    yaxis='y2'
)

# define layout with two y-axes
layout = go.Layout(
    title='2021-2022 Expenses',
    yaxis=dict(
        title='Expenses'
    ),
    yaxis2=dict(
        title='% Change',
        overlaying='y',
        side='right'
    )
)

# create figure object
fig1 = go.Figure(data=[trace1, trace2], layout=layout)

# show the figure
fig1.show()
Expenses2021_2022 = json.dumps(fig1, cls=plotly.utils.PlotlyJSONEncoder)


In [9]:
# 2022-2023 Data
filter2022_2023 = result[result['Year'] == '2022-2023']

trace1 = go.Bar(
    x=filter2022_2023.index,
    y=filter2022_2023['Grand Total'],
    name='Expenses'
)

# create trace for second y-axis
trace2 = go.Scatter(
    x=result.index,
    y=filter2022_2023['% Change'],
    name='% Change',
    yaxis='y2'
)

# define layout with two y-axes
layout = go.Layout(
    title='2022-2023 Expenses',
    yaxis=dict(
        title='Expenses'
    ),
    yaxis2=dict(
        title='% Change',
        overlaying='y',
        side='right'
    )
)

# create figure object
fig1 = go.Figure(data=[trace1, trace2], layout=layout)

# show the figure
fig1.show()
Expenses2022_2023 = json.dumps(fig1, cls=plotly.utils.PlotlyJSONEncoder)